<a href="https://colab.research.google.com/github/arthurabelo/Algebra-Linear-Python/blob/main/CalculadoraAlgebraLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculadora de Álgebra Linear: Um Estudo Didático

Este notebook tem como objetivo explicar, de forma didática, conceitos fundamentais de Álgebra Linear utilizando Python. Serão abordados:
- Bases e mudança de base
- Transformações lineares e suas matrizes associadas
- Cálculo do núcleo (kernel) e imagem de uma transformação
- Autovalores e autovetores

Cada seção traz explicações teóricas e exemplos práticos, permitindo a exploração interativa dos conceitos.

In [ ]:
import numpy as np
import re
from fractions import Fraction
from math import gcd


## 1. Definindo Bases e Mudança de Base

Em Álgebra Linear, uma **base** de um espaço vetorial é um conjunto de vetores linearmente independentes que geram todo o espaço. Qualquer vetor pode ser escrito como combinação linear dos vetores da base.

A **mudança de base** é o processo de converter as coordenadas de um vetor de uma base para outra. Isso é feito utilizando uma matriz de mudança de base.

A seguir, implementamos a classe `Base` para manipular bases e calcular a matriz de mudança de base.

In [ ]:
class Base:
    def __init__(self, vetores):
        self.vetores = np.array(vetores, dtype=float)
        self.dim = self.vetores.shape[1] if len(self.vetores.shape) > 1 else 1

    def matriz_para_base(self, nova_base):
        """
        Retorna a matriz de mudança de base de self para nova_base.
        A matriz de mudança de base permite converter coordenadas de uma base para outra.
        """
        if np.linalg.matrix_rank(self.vetores) < self.vetores.shape[1] or np.linalg.matrix_rank(nova_base.vetores) < nova_base.vetores.shape[1]:
            raise ValueError("Uma das bases fornecidas não é invertível (vetores linearmente dependentes).")
        return np.linalg.inv(self.vetores) @ nova_base.vetores

# Exemplo prático:
# Base padrão de R^3
base_padrao = Base(np.eye(3))
# Outra base de R^3
base2 = Base(np.array([[1,1,0],[0,1,1],[1,0,1]]).T)
# Matriz de mudança de base da base padrão para base2
P = base_padrao.matriz_para_base(base2)
print("Matriz de mudança de base (padrão -> base2):\n", P)

Matriz de mudança de base (padrão -> base2):
 [[1. 0. 1.]
 [1. 1. 0.]
 [0. 1. 1.]]


## 2. Transformações Lineares e Matrizes Associadas

Uma **transformação linear** é uma função entre espaços vetoriais que preserva operações de adição e multiplicação por escalar. Toda transformação linear pode ser representada por uma matriz, chamada **matriz associada**.

A seguir, implementamos a classe `TransformacaoLinear`, que interpreta uma transformação linear a partir de sua expressão simbólica e constrói sua matriz associada.

In [ ]:
class TransformacaoLinear:
    def __init__(self, incognitas, expressoes):
        self.incognitas = incognitas
        self.expressoes = expressoes
        self.matriz = self.matriz_associada(incognitas, expressoes)

    def matriz_associada(self, incognitas, expressoes):
        matriz = []
        for expr in expressoes:
            matriz.append(self.expr_para_coef(expr, incognitas))
        return np.array(matriz)

    @staticmethod
    def parse_transformacao(transformacao):
        def extrair_lista(regex, texto):
            match = re.search(regex, texto)
            if not match:
                return []
            lista_str = match.group(1)
            lista = []
            for item in lista_str.split(','):
                item = item.strip()
                if item == '':
                    continue
                lista.append(item)
            return lista
        try:
            incognitas = extrair_lista(r'T\s*\(\s*([^)]*?)\s*\)\s*', transformacao)
            expressoes = extrair_lista(r'=\s*\(\s*([^)]*?)\s*\)\s*', transformacao)
            if not incognitas or not expressoes:
                raise ValueError
            return incognitas, expressoes
        except Exception:
            print("Erro ao interpretar a transformação linear. Verifique o formato.")
            return [], []

    @staticmethod
    def expr_para_coef(expr, incognitas):
        coefs = [0.0 for _ in incognitas]
        expr = expr.replace(' ', '')
        termos = re.findall(r'[+-]?[^+-]+', expr)
        for termo in termos:
            if termo == '':
                continue
            for i, var in enumerate(incognitas):
                if var in termo:
                    coef = termo.replace(var, '')
                    if coef in ['', '+']:
                        coefs[i] += 1.0
                    elif coef == '-':
                        coefs[i] -= 1.0
                    else:
                        coefs[i] += float(coef)
        return coefs

# Exemplo prático:
# T(x, y, z) = (x-y, x+y, 3x+y)
incognitas, expressoes = TransformacaoLinear.parse_transformacao('T(x,y,z)=(x-y,x+y,3x+y)')
T = TransformacaoLinear(incognitas, expressoes)
print("Matriz associada à transformação T:")
print(T.matriz)

Matriz associada à transformação T:
[[ 1. -1.  0.]
 [ 1.  1.  0.]
 [ 3.  1.  0.]]


## 3. Núcleo e Imagem de uma Transformação Linear

O **núcleo** (ou kernel) de uma transformação linear T é o conjunto de todos os vetores que são enviados para o vetor nulo. Já a **imagem** é o conjunto de todos os vetores que podem ser obtidos como T(v) para algum v do domínio.

O núcleo indica as soluções do sistema homogêneo T(v) = 0, enquanto a imagem representa o "alcance" da transformação.

A seguir, implementamos métodos para calcular núcleo e imagem de uma transformação.

In [ ]:
def eliminacao_gauss(M):
    M = [row[:] for row in M]
    n = len(M)
    m = len(M[0]) - 1
    for i in range(n):
        max_row = i
        for k in range(i+1, n):
            if abs(M[k][i]) > abs(M[max_row][i]):
                max_row = k
        if abs(M[max_row][i]) < 1e-12:
            continue
        M[i], M[max_row] = M[max_row], M[i]
        for k in range(i+1, n):
            if abs(M[i][i]) < 1e-12:
                continue
            f = M[k][i] / M[i][i]
            for j in range(i, m+1):
                M[k][j] -= f * M[i][j]
    pivots = []
    for i in range(n):
        for j in range(m):
            if abs(M[i][j]) > 1e-10:
                pivots.append(j)
                break
    free_vars = [j for j in range(m) if j not in pivots]
    if not free_vars:
        return []
    base = []
    for free in free_vars:
        vec = [0.0]*m
        vec[free] = 1.0
        for i in reversed(range(len(pivots))):
            linhai = M[i]
            s = 0.0
            for j in free_vars:
                s += linhai[j]*vec[j]
            if abs(linhai[pivots[i]]) > 1e-10:
                vec[pivots[i]] = -s/linhai[pivots[i]]
        base.append(vec)
    return base

def nucleo(matriz):
    A = np.array(matriz, dtype=float)
    n_linhas, n_colunas = A.shape
    M = np.hstack([A, np.zeros((n_linhas, 1))])
    return eliminacao_gauss(M.tolist())

def gauss_jordan(m):
    n_rows = len(m)
    n_cols = len(m[0])
    a = [row[:] for row in m]
    min_dim = min(n_rows, n_cols)
    row_used = [False]*n_rows
    for i in range(min_dim):
        max_row = None
        for r in range(i, n_rows):
            if abs(a[r][i]) > 1e-12 and not row_used[r]:
                max_row = r
                break
        if max_row is None:
            continue
        if max_row != i:
            a[i], a[max_row] = a[max_row], a[i]
        row_used[i] = True
        for j in range(n_rows):
            if (i != j):
                if abs(a[i][i]) < 1e-12:
                    continue
                p = a[j][i] / a[i][i]
                for k in range(n_cols):
                    a[j][k] -= a[i][k] * p
    for i in range(min_dim):
        if abs(a[i][i]) > 1e-12:
            div = a[i][i]
            for k in range(n_cols):
                a[i][k] /= div
    return a

def imagem(matriz):
    A = np.array(matriz, dtype=float)
    Aj = gauss_jordan(A.T.tolist())
    Aj = np.array(Aj).T
    base = []
    for j in range(Aj.shape[1]):
        col = Aj[:, j]
        if np.any(np.abs(col) > 1e-10):
            base.append([int(round(x)) if abs(x - round(x)) < 1e-10 else x for x in col])
    return base

# Exemplo prático:
print("\nBase do núcleo de T:")
print(nucleo(T.matriz))
print("Base da imagem de T:")
print(imagem(T.matriz))


Base do núcleo de T:
[[-0.0, 0.0, 1.0]]
Base da imagem de T:
[[1, 0, 1], [0, 1, 2]]


## 4. Autovalores e Autovetores

Dada uma matriz quadrada $A$, um **autovalor** $\lambda$ e um **autovetor** $v$ satisfazem $A v = \lambda v$. Autovalores e autovetores são fundamentais para entender o comportamento de transformações lineares, especialmente em aplicações como sistemas dinâmicos, diagonalização e muito mais.

A seguir, implementamos métodos para calcular autovalores e autovetores de uma matriz.

In [ ]:
def polinomio_caracteristico(matriz):
    matriz = np.array(matriz)
    n = matriz.shape[0]
    assert matriz.shape[1] == n
    coeficientes = np.array([1.])
    matriz_atual = np.array(matriz)
    for k in range(1, n + 1):
        coef = -np.trace(matriz_atual) / k
        coeficientes = np.append(coeficientes, coef)
        matriz_atual += np.diag(np.repeat(coef, n))
        matriz_atual = matriz @ matriz_atual
    return coeficientes

def calcula_autovalores(matriz, coeficientes):
    autovalores = np.roots(coeficientes)
    autovalores = np.array([val.real if abs(val.imag) < 1e-10 else val for val in autovalores])
    return autovalores

def calcula_autovetores(matriz, autovalores):
    A = np.array(matriz, dtype=float)
    n, m = A.shape
    I = np.eye(n)
    autovetores = []
    for val in autovalores:
        if np.iscomplex(val) and abs(val.imag) > 1e-10:
            autovetores.append(np.zeros(n))
            continue
        M = A - val * I
        base_nucleo = nucleo(M)
        vec = None
        if base_nucleo and np.linalg.norm(base_nucleo[0]) > 1e-12:
            vec = np.array(base_nucleo[0], dtype=float)
        else:
            u, s, vh = np.linalg.svd(M)
            vec = vh[-1, :]
        if vec is not None and np.linalg.norm(vec) > 1e-12:
            vec = vec / np.linalg.norm(vec)
            autovetores.append(vec.real)
        else:
            autovetores.append(np.zeros(n))
    return np.array(autovetores).T

# Exemplo prático (apenas para matrizes quadradas):
if T.matriz.shape[0] == T.matriz.shape[1]:
    coef = polinomio_caracteristico(T.matriz)
    print("Polinômio característico:", coef)
    autovalores = calcula_autovalores(T.matriz, coef)
    print("Autovalores:", autovalores)
    autovetores = calcula_autovetores(T.matriz, autovalores)
    print("Autovetores (colunas):\n", autovetores)
else:
    print("A matriz associada não é quadrada. Não há autovalores/autovetores.")

Polinômio característico: [ 1. -2.  2. -0.]
Autovalores: [1.+1.j 1.-1.j 0.+0.j]
Autovetores (colunas):
 [[ 0.  0. -0.]
 [ 0.  0.  0.]
 [ 0.  0.  1.]]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_12604\176840292.py:43: ComplexWarning: Casting complex values to real discards the imaginary part
  A = np.array(matriz, dtype=float)


## 5. Menu Interativo para Exploração de Transformações Lineares

A seguir, apresentamos um menu interativo para que você possa explorar os conceitos apresentados:
- Defina uma transformação linear
- Calcule núcleo, imagem, autovalores e autovetores
- Experimente diferentes bases

**Execute a célula abaixo e siga as instruções no console do notebook.**

In [ ]:
def menu_interativo():
    print("Menu:")
    print("1. Calcular base do núcleo e base da imagem da transformação linear")
    print("2. Calcular matriz da transformação em bases distintas")
    print("3. Calcular autovalores e autovetores da transformação linear")
    print("4. Alterar transformação linear")
    print("0. Sair")
    return input("Escolha uma opção: ")

def main_notebook():
    print("Exemplo: T(x,y,z)=(x-y,x+y,3x+y)")
    transformacao = input("Digite a transformação linear: ")
    incognitas, expressoes = TransformacaoLinear.parse_transformacao(transformacao)
    T = TransformacaoLinear(incognitas, expressoes)
    print("Matriz associada:")
    print(T.matriz)
    while True:
        op = menu_interativo()
        if op == '1':
            base_nucleo = nucleo(T.matriz)
            print("Ker(T) (base do núcleo):", base_nucleo)
            print("Dimensão do núcleo:", len(base_nucleo))
            base_imagem = imagem(T.matriz)
            print("Im(T) (base da imagem):", base_imagem)
            print("Dimensão da imagem:", len(base_imagem))
        elif op == '2':
            print("Digite a base do domínio (ex: 1,0,0;0,1,0;0,0,1):")
            base_dom = input()
            vetores_dom = [list(map(float, v.split(','))) for v in base_dom.strip().split(';') if v.strip() != '']
            print("Digite a base do contradomínio (mesmo formato):")
            base_cod = input()
            vetores_cod = [list(map(float, v.split(','))) for v in base_cod.strip().split(';') if v.strip() != '']
            B_dom = Base(np.array(vetores_dom).T)
            B_cod = Base(np.array(vetores_cod).T)
            matriz_nova = B_cod.matriz_para_base(B_dom)
            print("Matriz da transformação nas novas bases:")
            print(matriz_nova)
        elif op == '3':
            if T.matriz.shape[0] != T.matriz.shape[1]:
                print("A matriz associada não é quadrada. Autovalores e autovetores só existem para matrizes quadradas.")
                continue
            coef = polinomio_caracteristico(T.matriz)
            print("Polinômio característico:", coef)
            autovalores = calcula_autovalores(T.matriz, coef)
            print("Autovalores:", autovalores)
            autovetores = calcula_autovetores(T.matriz, autovalores)
            print("Autovetores (colunas):\n", autovetores)
        elif op == '4':
            transformacao = input("Digite a nova transformação linear: ")
            incognitas, expressoes = TransformacaoLinear.parse_transformacao(transformacao)
            T = TransformacaoLinear(incognitas, expressoes)
            print("Matriz associada:")
            print(T.matriz)
        elif op == '0':
            print("Saindo...")
            break
        else:
            print("Opção inválida!")

# Para rodar o menu interativo, descomente a linha abaixo:
# main_notebook()

Exemplo: T(x,y,z)=(x-y,x+y,3x+y)
Matriz associada:
[[1. 2. 0. 0.]
 [0. 3. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 5. 3.]]
Menu:
1. Calcular base do núcleo e base da imagem da transformação linear
2. Calcular matriz da transformação em bases distintas
3. Calcular autovalores e autovetores da transformação linear
4. Alterar transformação linear
0. Sair
Matriz associada:
[[1. 2. 0. 0.]
 [0. 3. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 5. 3.]]
Menu:
1. Calcular base do núcleo e base da imagem da transformação linear
2. Calcular matriz da transformação em bases distintas
3. Calcular autovalores e autovetores da transformação linear
4. Alterar transformação linear
0. Sair
Ker(T) (base do núcleo): []
Dimensão do núcleo: 0
Im(T) (base da imagem): [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
Dimensão da imagem: 4
Menu:
1. Calcular base do núcleo e base da imagem da transformação linear
2. Calcular matriz da transformação em bases distintas
3. Calcular autovalores e autovetores da transformação linear
4. 

### Explicação do Algoritmo de Mudança de Base

O algoritmo de mudança de base implementado na classe `Base` utiliza conceitos fundamentais de Álgebra Linear:
- **Bases** são conjuntos de vetores linearmente independentes que geram o espaço vetorial.
- Para converter as coordenadas de um vetor de uma base para outra, usamos a **matriz de mudança de base**.

**Como funciona o algoritmo:**
1. Verifica se as duas bases são realmente bases (ou seja, se seus vetores são linearmente independentes e formam uma matriz invertível).
2. Calcula a inversa da matriz da base original.
3. Multiplica a inversa da base original pela matriz da nova base: $P = B^{-1}C$, onde $B$ é a matriz da base original e $C$ da nova base.
4. O resultado é a matriz que transforma coordenadas da base original para a nova base.

Esse processo é essencial para resolver problemas em que precisamos comparar representações de vetores ou transformações em diferentes bases.

### Explicação do Algoritmo de Construção da Matriz Associada

A matriz associada a uma transformação linear é construída a partir das expressões que definem a transformação. O algoritmo segue os seguintes passos:
1. **Leitura das variáveis e expressões:**
   - O usuário fornece a transformação no formato $T(x, y, z) = (x-y, x+y, 3x+y)$.
   - O algoritmo extrai as incógnitas (variáveis) e as expressões de saída.
2. **Construção da matriz:**
   - Para cada expressão de saída, o algoritmo identifica o coeficiente de cada variável.
   - Esses coeficientes formam as linhas da matriz associada.
3. **Resultado:**
   - A matriz resultante representa a transformação linear em relação à base padrão.

Esse método automatiza a passagem de uma descrição simbólica para uma representação matricial, fundamental para cálculos em Álgebra Linear.

### Explicação dos Algoritmos de Núcleo (Kernel) e Imagem

#### Núcleo (Kernel)
O núcleo de uma transformação linear $T$ é o conjunto de vetores $v$ tal que $T(v) = 0$. Para encontrar uma base do núcleo:
1. Monta-se o sistema homogêneo $A\vec{x} = 0$, onde $A$ é a matriz associada.
2. O algoritmo aplica a **eliminação de Gauss** para escalonar a matriz aumentada.
3. Identifica as variáveis livres e constrói vetores que formam uma base para o espaço solução.

#### Imagem
A imagem é o subespaço gerado pelas colunas da matriz associada:
1. O algoritmo aplica o método de **Gauss-Jordan** para obter uma matriz reduzida por linhas.
2. As colunas não nulas da matriz reduzida formam uma base para a imagem.

Esses algoritmos são essenciais para entender as propriedades fundamentais de uma transformação linear, como sua injetividade e sobrejetividade.

### Explicação do Algoritmo de Autovalores e Autovetores

#### Autovalores
O cálculo dos autovalores é feito pelo **método de Leverrier-Faddeev**, que evita o cálculo direto do determinante:
1. Constrói-se iterativamente matrizes auxiliares e calcula-se o traço (soma dos elementos da diagonal principal).
2. Os coeficientes do polinômio característico $\det(A - \lambda I)$ são obtidos sem expandir determinantes.
3. As raízes desse polinômio são os autovalores.

#### Autovetores
Para cada autovalor $\lambda$:
1. Calcula-se a matriz $A - \lambda I$.
2. O núcleo dessa matriz é encontrado (usando eliminação de Gauss), e qualquer vetor não nulo desse núcleo é um autovetor associado.
3. Se necessário, utiliza-se SVD para garantir um vetor não nulo.

Esses algoritmos são fundamentais para análise espectral de matrizes, com aplicações em estabilidade, sistemas dinâmicos, física e muito mais.

### Explicação do Menu Interativo

O menu interativo foi projetado para facilitar a exploração dos conceitos de Álgebra Linear de forma prática:
- Permite ao usuário definir diferentes transformações lineares e experimentar com bases distintas.
- Calcula e exibe, de forma automática, núcleo, imagem, autovalores e autovetores.
- O código foi adaptado para rodar em ambiente de notebook, tornando o aprendizado mais dinâmico e visual.

Essa abordagem é ideal para apresentações e seminários, pois permite demonstrar ao vivo como os algoritmos funcionam e como os conceitos se relacionam na prática.